# Baseline Model For Anomaly Detection

# Summary

- Introduction
- Before running the notebook
- Config
- Run the evaluation

## Introduction

This is the code used to run the evaluation on all models and all datasets so you can compare the results.
The project output is as follow:
- [stats_for_website.json](https://drive.google.com/file/d/1mBRRRhWLsV2pCC-GMst-9KQ65Owj3jXH/view?usp=sharing) - File used to display the stats on the dataset explorer


- [benchmarks.json](https://drive.google.com/file/d/1cBk0Yd2smMMskUp4U7lPeS2lM2tFPapC/view?usp=sharing) - File used to display the benchmark on the dataset explorer


- In the dataset, duplicate each pictures with it's inference results from efficientad, including the heatmpa. Each file has a `_result` in the filename. Used on the table in the dataset explorer. All the images on this have been reduced to 300x300. You can download the results.zip [here](https://drive.google.com/file/d/1nSSYSPowrj8FwpAAX5cLDZEQldFVf9Uk/view?usp=sharing).

Here is the command used if you are interested: 
```sh
find . -type f \( -name "*.jpg" -o -name "*.png" \) -exec sips -Z 300 {} \;
```

## Before running the notebook

Let's prepare the environement before running the notebook.

Make sure you have run the notebooks `1_baseline`, `2_efficientad`, `3_fomoad` before running this!

Run the following command in the terminal from the **`ai`** folder.

```sh
conda create -p envs/evaluate python=3.11.7 -y
conda activate envs/evaluate

pip install -r requirements/evaluate.txt
```

## Config

In [ ]:
import sys  # Import the sys module, required to access system-specific parameters and functions  
sys.path.append('../code/')  # Add the '../code/' directory to the path that Python looks in for files to import  
  
from evaluate import * # Import all contents of the evaluate module

In [2]:

config = {
    "dataset_path": "datasets",
    "baseline" : {
        "cookies_1": "../output/baseline/cookies_1",
        "cookies_2": "../output/baseline/cookies_2",
        "cookies_3": "../output/baseline/cookies_3",
    },
    "efficientad" : {
        "cookies_1": "../output/efficientad/cookies_1",
        "cookies_2": "../output/efficientad/cookies_2",
        "cookies_3": "../output/efficientad/cookies_3",
    },
    "fomoad" : { # Put both mac and linux
        "cookies_1": "../output/fomoad/cookies_1",
        "cookies_1_threshold": 3.7,
        "cookies_2": "../output/fomoad/cookies_2",
        "cookies_2_threshold": 5.2,
        "cookies_3": "../output/fomoad/cookies_3",
        "cookies_3_threshold": 5.0,
    },
}

## Run the evaluation

In [3]:
def load_dataset(subdataset):
    no_anomaly_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/no_anomaly/*.jpg")  # adjust the pattern if needed

    anomaly_lvl_1_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/anomaly_lvl_1/*.jpg")
    anomaly_lvl_2_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/anomaly_lvl_2/*.jpg")
    anomaly_lvl_3_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/anomaly_lvl_3/*.jpg")

    no_anomaly_labels = ["no_anomaly"] * len(no_anomaly_paths)
    no_anomaly_difficulty = ["na"] * len(no_anomaly_paths)

    anomaly_1_labels = ["anomaly"] * len(anomaly_lvl_1_paths)
    anomaly_1_difficulty = ["easy"] * len(anomaly_lvl_1_paths)
    
    anomaly_2_labels = ["anomaly"] * len(anomaly_lvl_2_paths)
    anomaly_2_difficulty = ["medium"] * len(anomaly_lvl_2_paths)
    
    anomaly_3_labels = ["anomaly"] * len(anomaly_lvl_3_paths)
    anomaly_3_difficulty = ["hard"] * len(anomaly_lvl_3_paths)

    no_anomaly_df = pd.DataFrame({'filename': no_anomaly_paths, 'class': no_anomaly_labels, 'difficulty': no_anomaly_difficulty})  

    anomaly_1_df = pd.DataFrame({'filename': anomaly_lvl_1_paths, 'class': anomaly_1_labels, 'difficulty': anomaly_1_difficulty})  
    anomaly_2_df = pd.DataFrame({'filename': anomaly_lvl_2_paths, 'class': anomaly_2_labels, 'difficulty': anomaly_2_difficulty})  
    anomaly_3_df = pd.DataFrame({'filename': anomaly_lvl_3_paths, 'class': anomaly_3_labels, 'difficulty': anomaly_3_difficulty})
    
    combined_df = pd.concat([no_anomaly_df, anomaly_1_df, anomaly_2_df, anomaly_3_df])
    return combined_df.reset_index(drop=True)  

In [4]:
def load_baseline(dataset_name):
    baseline_model = tf.keras.models.load_model(f'{config["baseline"][dataset_name]}/local.keras')
    edge_impulse_model_converted = ImageImpulseRunner(f'{config["baseline"][dataset_name]}/ei.eim')
    edge_impulse_model_converted.init()
        
    return baseline_model, edge_impulse_model_converted

def evaluate_baseline(baseline_model, edge_impulse_model_converted, item):
    cl, time, memory, score = inference_baseline(baseline_model, item["filename"])
    cl_ei, time_ei, memory_ei, score_ei = inference_baseline_ei(edge_impulse_model_converted, item["filename"])
    return cl, time, memory, score, cl_ei, time_ei, memory_ei, score_ei

In [5]:
def load_efficientad(dataset_name):
    config_efficient_ad = {
        "seed": 42,
        "on_gpu": torch.cuda.is_available(),
        
        "out_channels": 384,
        "image_size": 256,
    }
    
    efficientad_models = torch.load(f'{config["efficientad"][dataset_name]}/all_models.pth', map_location=torch.device('cpu'))
    map_normalization = torch.load(f'{config["efficientad"][dataset_name]}/map_normalization.pth', map_location=torch.device('cpu')) 
    
    with open(f'{config["efficientad"][dataset_name]}/best_threshold.pkl', 'rb') as file:  
        best_threshold = pickle.load(file) 
     
    efficientad = EfficientADInference(
        config=config_efficient_ad,
        models=efficientad_models,
        map_normalization=map_normalization,
        threshold=best_threshold,
    )
    
    return efficientad

def evaluate_efficientad(efficientad, item):
    cl, time, memory, score = inference_efficientad(efficientad, item["filename"])
    return cl, time, memory, score

In [6]:
def load_fomoad(dataset_name):
    edge_impulse_model_fomoad = ImageImpulseRunner(f'{config["fomoad"][dataset_name]}/fomoad_runner-mac-x86_64.eim')
    edge_impulse_model_fomoad.init()
     
    return edge_impulse_model_fomoad

def evaluate_fomoad(edge_impulse_model_fomoad, item, threshold):
    cl, time, memory, score = inference_fomoad_ei(edge_impulse_model_fomoad, item["filename"], threshold)
    return cl, time, memory, score


In [7]:
def evaluate_one_dataset(dataset_name):
    print(f"Evaluating {dataset_name}")
    
    # Loading all model
    baseline_model, ei_model_converted = load_baseline(dataset_name)
    efficientad = load_efficientad(dataset_name)
    ei_model_fomoad = load_fomoad(dataset_name)
    
    # Loading dataset
    dataset = load_dataset(dataset_name)
    
    # Looping through the full dataset
    for index, row in tqdm(dataset.iterrows(), total=len(dataset)):
        # Evaluate each model
        cl_baseline, time_baseline, memory_baseline, score_baseline, cl_ei_baseline, time_ei_baseline, memory_ei_baseline, score_ei_baseline = evaluate_baseline(baseline_model, ei_model_converted, row)
        
        cl_efficientad, time_efficientad, memory_efficientad, score_efficientad = evaluate_efficientad(efficientad, row)
        
        cl_fomodad, time_fomoad, memory_fomoad, score_fomoad = evaluate_fomoad(ei_model_fomoad, row, config["fomoad"][f"{dataset_name}_threshold"])
        
        # Update the dataframe to add the different scroes
        dataset.at[index, 'cl_baseline'] = cl_baseline
        dataset.at[index, 'time_baseline'] = time_baseline
        dataset.at[index, 'memory_baseline'] = memory_baseline
        dataset.at[index, 'score_baseline'] = score_baseline
        
        dataset.at[index, 'cl_ei_baseline'] = cl_ei_baseline
        dataset.at[index, 'time_ei_baseline'] = time_ei_baseline
        dataset.at[index, 'memory_ei_baseline'] = memory_ei_baseline
        dataset.at[index, 'score_ei_baseline'] = score_ei_baseline
        
        dataset.at[index, 'cl_efficientad'] = cl_efficientad
        dataset.at[index, 'time_efficientad'] = time_efficientad
        dataset.at[index, 'memory_efficientad'] = memory_efficientad
        dataset.at[index, 'score_efficientad'] = score_efficientad
        
        dataset.at[index, 'cl_fomodad'] = cl_fomodad
        dataset.at[index, 'time_fomoad'] = time_fomoad
        dataset.at[index, 'memory_fomoad'] = memory_fomoad
        dataset.at[index, 'score_fomoad'] = score_fomoad
        
        save_path = row["filename"].replace('.jpg', '_result.jpg')  
        dataset.at[index, 'result_img_path'] = save_path
    return dataset

In [8]:
dataset_1 = evaluate_one_dataset("cookies_1")

Evaluating cookies_1
- Setting seed to 42
- OK - Setting seed to 42 (3.41 ms)



  0%|          | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 213ms/step


  0%|          | 1/200 [00:00<02:42,  1.23it/s]

1/1 [==============================] - 0s 30ms/step


...

 99%|█████████▉| 198/200 [01:22<00:00,  2.38it/s]

1/1 [==============================] - 0s 19ms/step


100%|█████████▉| 199/200 [01:22<00:00,  2.30it/s]

1/1 [==============================] - 0s 18ms/step


100%|██████████| 200/200 [01:22<00:00,  2.41it/s]


In [9]:
dataset_2 = evaluate_one_dataset("cookies_2")

Evaluating cookies_2
- Setting seed to 42
- OK - Setting seed to 42 (1.05 ms)



  0%|          | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 163ms/step


  0%|          | 1/200 [00:00<02:07,  1.57it/s]

1/1 [==============================] - 0s 32ms/step


...

 99%|█████████▉| 198/200 [01:29<00:00,  2.31it/s]

1/1 [==============================] - 0s 22ms/step


100%|█████████▉| 199/200 [01:29<00:00,  2.24it/s]

1/1 [==============================] - 0s 21ms/step


100%|██████████| 200/200 [01:30<00:00,  2.22it/s]


In [10]:
dataset_3 = evaluate_one_dataset("cookies_3")

Evaluating cookies_3
- Setting seed to 42
- OK - Setting seed to 42 (1.42 ms)



  0%|          | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 175ms/step


  0%|          | 1/200 [00:00<02:19,  1.43it/s]

1/1 [==============================] - 0s 42ms/step


...

 99%|█████████▉| 198/200 [01:28<00:00,  2.27it/s]

1/1 [==============================] - 0s 21ms/step


100%|█████████▉| 199/200 [01:29<00:00,  2.32it/s]

1/1 [==============================] - 0s 22ms/step


100%|██████████| 200/200 [01:29<00:00,  2.23it/s]


In [11]:
dataset_3

,filename,class,difficulty,cl_baseline,time_baseline,memory_baseline,score_baseline,cl_ei_baseline,time_ei_baseline,memory_ei_baseline,score_ei_baseline,cl_efficientad,time_efficientad,memory_efficientad,score_efficientad,cl_fomodad,time_fomoad,memory_fomoad,score_fomoad,result_img_path
0,datasets/cookies_3/no_anomaly/20240417_134027.jpg,no_anomaly,na,no_anomaly,215.291262,11.968750,0.872619,no_anomaly,74.587107,1.765625,0.790563,no_anomaly,330.019951,-2176.0,0.196108,no_anomaly,73.952198,-23.515625,3.384082,datasets/cookies_3/no_anomaly/20240417_134027_...
1,datasets/cookies_3/no_anomaly/20240417_134225.jpg,no_anomaly,na,no_anomaly,79.087973,0.156250,0.815615,no_anomaly,44.698000,0.281250,0.836219,no_anomaly,306.039095,7424.0,0.062137,no_anomaly,38.233995,4.078125,2.895372,datasets/cookies_3/no_anomaly/20240417_134225_...
2,datasets/cookies_3/no_anomaly/20240417_134351.jpg,no_anomaly,na,no_anomaly,42.485952,0.046875,0.831380,no_anomaly,33.839941,1.203125,0.623720,no_anomaly,313.975096,13488.0,0.063899,no_anomaly,36.097050,3.046875,3.439135,datasets/cookies_3/no_anomaly/20240417_134351_...
3,datasets/cookies_3/no_anomaly/20240417_134353.jpg,no_anomaly,na,no_anomaly,41.754246,0.000000,0.828037,no_anomaly,33.387184,1.203125,0.633182,no_anomaly,320.642233,-31632.0,0.047855,no_anomaly,36.247015,3.109375,3.617379,datasets/cookies_3/no_anomaly/20240417_134353_...
4,datasets/cookies_3/no_anomaly/20240417_134227.jpg,no_anomaly,na,no_anomaly,39.963245,0.171875,0.682166,no_anomaly,32.882929,1.250000,0.871255,no_anomaly,311.959028,16352.0,0.059206,no_anomaly,38.475990,2.156250,3.650349,datasets/cookies_3/no_anomaly/20240417_134227_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,datasets/cookies_3/anomaly_lvl_3/20240417_1405...,anomaly,hard,no_anomaly,45.591116,0.187500,0.593224,anomaly,34.983873,0.000000,0.677545,anomaly,299.077988,1936.0,0.953776,anomaly,37.696123,0.093750,13.224107,datasets/cookies_3/anomaly_lvl_3/20240417_1405...
196,datasets/cookies_3/anomaly_lvl_3/20240417_1406...,anomaly,hard,no_anomaly,100.214720,0.109375,0.648835,no_anomaly,40.426970,0.000000,0.539894,anomaly,312.687159,2736.0,0.426243,anomaly,36.779881,0.203125,5.771534,datasets/cookies_3/anomaly_lvl_3/20240417_1406...
197,datasets/cookies_3/anomaly_lvl_3/20240417_1405...,anomaly,hard,no_anomaly,43.220043,0.328125,0.816310,no_anomaly,33.733130,0.000000,0.571221,anomaly,290.580273,9408.0,0.455816,anomaly,36.035061,0.234375,5.345970,datasets/cookies_3/anomaly_lvl_3/20240417_1405...
198,datasets/cookies_3/anomaly_lvl_3/20240417_1405...,anomaly,hard,no_anomaly,43.244123,0.046875,0.522906,anomaly,36.174059,0.000000,0.653283,anomaly,294.780016,0.0,0.501725,anomaly,36.408663,0.109375,8.528876,datasets/cookies_3/anomaly_lvl_3/20240417_1405...


In [12]:
dataset_2['cl_efficientad'].value_counts()

cl_efficientad
anomaly       101
no_anomaly     99
Name: count, dtype: int64

In [13]:
def get_score(dataset, name):  
    row = [name]  
      
    # For Baseline  
    f1_all_baseline = f1_score(dataset['class'], dataset['cl_baseline'], pos_label='anomaly')  
    f1_all_baseline_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_baseline'], pos_label='no_anomaly')  
    f1_all_baseline_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_baseline'], pos_label='anomaly')  
    f1_all_baseline_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_baseline'], pos_label='anomaly')  
    f1_all_baseline_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_baseline'], pos_label='anomaly')  
    row.extend([name, f1_all_baseline, f1_all_baseline_na, f1_all_baseline_lvl1, f1_all_baseline_lvl2, f1_all_baseline_lvl3,  
                dataset['time_baseline'].mean(), dataset['memory_baseline'].mean()])  
  
    # For Baseline EI  
    f1_all_baseline_ei = f1_score(dataset['class'], dataset['cl_ei_baseline'], pos_label='anomaly')  
    f1_all_baseline_ei_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_ei_baseline'], pos_label='no_anomaly')  
    f1_all_baseline_ei_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_ei_baseline'], pos_label='anomaly')  
    f1_all_baseline_ei_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_ei_baseline'], pos_label='anomaly')  
    f1_all_baseline_ei_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_ei_baseline'], pos_label='anomaly')  
    row.extend([name, f1_all_baseline_ei, f1_all_baseline_ei_na, f1_all_baseline_ei_lvl1, f1_all_baseline_ei_lvl2, f1_all_baseline_ei_lvl3,  
                dataset['time_ei_baseline'].mean(), dataset['memory_ei_baseline'].mean()])  
  
    # For EfficientAD  
    f1_all_efficientad = f1_score(dataset['class'], dataset['cl_efficientad'], pos_label='anomaly')  
    f1_all_efficientad_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_efficientad'], pos_label='no_anomaly')  
    f1_all_efficientad_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_efficientad'], pos_label='anomaly')  
    f1_all_efficientad_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_efficientad'], pos_label='anomaly')  
    f1_all_efficientad_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_efficientad'], pos_label='anomaly')  
    row.extend([name, f1_all_efficientad, f1_all_efficientad_na, f1_all_efficientad_lvl1, f1_all_efficientad_lvl2, f1_all_efficientad_lvl3,  
                dataset['time_efficientad'].mean(), dataset['memory_efficientad'].mean()])  
  
    # For FOMOAD  
    f1_all_fomoad = f1_score(dataset['class'], dataset['cl_fomodad'], pos_label='anomaly')  
    f1_all_fomoad_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_fomodad'], pos_label='no_anomaly')  
    f1_all_fomoad_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_fomodad'], pos_label='anomaly')  
    f1_all_fomoad_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_fomodad'], pos_label='anomaly')  
    f1_all_fomoad_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_fomodad'], pos_label='anomaly')  
    row.extend([name, f1_all_fomoad, f1_all_fomoad_na, f1_all_fomoad_lvl1, f1_all_fomoad_lvl2, f1_all_fomoad_lvl3,  
                dataset['time_fomoad'].mean(), dataset['memory_fomoad'].mean()])  
  
    return row

benchmarks = pd.DataFrame(columns=["Dataset", 'Baseline Dataset', 'Baseline F1 ALL', 'Baseline F1 NO ANOMALY', 'Baseline F1 EASY', 'Baseline F1 MEDIUM', 'Baseline F1 HARD', 'Baseline Time (ms)', 'Baseline RAM (KB)',  
                                   'Baseline EI Dataset', 'Baseline EI F1 ALL', 'Baseline EI F1 NO ANOMALY', 'Baseline EI F1 EASY', 'Baseline EI F1 MEDIUM', 'Baseline EI F1 HARD', 'Baseline EI Time (ms)', 'Baseline EI RAM (KB)',  
                                   'EfficientAD Dataset', 'EfficientAD F1 ALL', 'EfficientAD F1 NO ANOMALY', 'EfficientAD F1 EASY', 'EfficientAD F1 MEDIUM', 'EfficientAD F1 HARD', 'EfficientAD Time (ms)', 'EfficientAD RAM (KB)',  
                                   'FOMOAD Dataset', 'FOMOAD F1 ALL', 'FOMOAD F1 NO ANOMALY', 'FOMOAD F1 EASY', 'FOMOAD F1 MEDIUM', 'FOMOAD F1 HARD', 'FOMOAD Time (ms)', 'FOMOAD RAM (KB)'])  
  
# Assuming dataset1, dataset2, dataset3 are your actual datasets  
results_dataset1 = pd.DataFrame([get_score(dataset_1, "cookies_1")], columns=benchmarks.columns)  
results_dataset2 = pd.DataFrame([get_score(dataset_2, "cookies_2")], columns=benchmarks.columns)  
results_dataset3 = pd.DataFrame([get_score(dataset_3, "cookies_3")], columns=benchmarks.columns)  

def dataframe_to_dict(df):  
    return df.to_dict(orient='records')[0]  

results = {  
    "cookies_1": dataframe_to_dict(results_dataset1),  
    "cookies_2": dataframe_to_dict(results_dataset2),  
    "cookies_3": dataframe_to_dict(results_dataset3)  
}

benchmarks = pd.concat([benchmarks, results_dataset1, results_dataset2, results_dataset3], ignore_index=True)
import json
with open('../output/benchmarks.json', 'w') as outfile:  
    outfile.write(json.dumps(results))

/var/folders/x_/0z24g8110_n09vlvhxmrh0_w0000gp/T/ipykernel_67880/2594109346.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  benchmarks = pd.concat([benchmarks, results_dataset1, results_dataset2, results_dataset3], ignore_index=True)


In [14]:
results_dataset1

,Dataset,Baseline Dataset,Baseline F1 ALL,Baseline F1 NO ANOMALY,Baseline F1 EASY,Baseline F1 MEDIUM,Baseline F1 HARD,Baseline Time (ms),Baseline RAM (KB),Baseline EI Dataset,...,EfficientAD Time (ms),EfficientAD RAM (KB),FOMOAD Dataset,FOMOAD F1 ALL,FOMOAD F1 NO ANOMALY,FOMOAD F1 EASY,FOMOAD F1 MEDIUM,FOMOAD F1 HARD,FOMOAD Time (ms),FOMOAD RAM (KB)
0,cookies_1,cookies_1,0.882682,1.0,1.0,1.0,0.275862,43.899704,0.132109,cookies_1,...,299.686815,8231.44,cookies_1,0.943005,0.989899,1.0,1.0,0.780488,34.033175,2.381328


In [15]:
new_columns = pd.MultiIndex.from_tuples([  
    ('', 'Dataset'),                                # Common dataset column  
    ('Baseline', 'Dataset'),  
    ('Baseline', 'F1 ALL'), ('Baseline', 'F1 NO ANOMALY'), ('Baseline', 'F1 EASY'), ('Baseline', 'F1 MEDIUM'), ('Baseline', 'F1 HARD'), ('Baseline', 'Time (ms)'), ('Baseline', 'RAM (KB)'),  
    ('Baseline EI', 'Dataset'),  
    ('Baseline EI', 'F1 ALL'), ('Baseline EI', 'F1 NO ANOMALY'), ('Baseline EI', 'F1 EASY'), ('Baseline EI', 'F1 MEDIUM'), ('Baseline EI', 'F1 HARD'), ('Baseline EI', 'Time (ms)'), ('Baseline EI', 'RAM (KB)'),  
    ('EfficientAD', 'Dataset'),  
    ('EfficientAD', 'F1 ALL'), ('EfficientAD', 'F1 NO ANOMALY'), ('EfficientAD', 'F1 EASY'), ('EfficientAD', 'F1 MEDIUM'), ('EfficientAD', 'F1 HARD'), ('EfficientAD', 'Time (ms)'), ('EfficientAD', 'RAM (KB)'),  
    ('FOMOAD', 'Dataset'),  
    ('FOMOAD', 'F1 ALL'), ('FOMOAD', 'F1 NO ANOMALY'), ('FOMOAD', 'F1 EASY'), ('FOMOAD', 'F1 MEDIUM'), ('FOMOAD', 'F1 HARD'), ('FOMOAD', 'Time (ms)'), ('FOMOAD', 'RAM (KB)')  
])  
benchmarks.columns = new_columns  

# Show styled DataFrame  
print("BASELINE")
display(benchmarks.loc[:, 'Baseline'])
print("Baseline EI")
display(benchmarks.loc[:, 'Baseline EI'])  
print("EfficientAD")
display(benchmarks.loc[:, 'EfficientAD'])
print("FOMO AD")
display(benchmarks.loc[:, 'FOMOAD'])

BASELINE


,Dataset,F1 ALL,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.882682,1.000000,1.0,1.0,0.275862,43.899704,0.132109
1,cookies_2,0.880435,0.984772,1.0,1.0,0.387097,48.814734,-0.298672
2,cookies_3,0.888889,1.000000,1.0,1.0,0.333333,47.018294,-0.634219


Baseline EI


,Dataset,F1 ALL,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.877778,0.994975,1.0,1.0,0.275862,35.320630,0.440625
1,cookies_2,0.878505,0.888889,1.0,1.0,0.863636,40.969303,-0.857500
2,cookies_3,0.930481,1.000000,1.0,1.0,0.648649,38.269688,-0.221016


EfficientAD


,Dataset,F1 ALL,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.931373,0.952880,1.0,1.0,0.888889,299.686815,8231.44
1,cookies_2,0.995025,0.994975,1.0,1.0,1.000000,319.951710,284.24
2,cookies_3,0.994975,1.000000,1.0,1.0,0.979592,319.854028,3025.52


FOMO AD


,Dataset,F1 ALL,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.943005,0.989899,1.0,1.0,0.780488,34.033175,2.381328
1,cookies_2,0.980198,0.984772,1.0,1.0,0.979592,38.634827,-0.465625
2,cookies_3,0.989899,1.000000,1.0,1.0,0.958333,40.206773,0.079219


In [16]:
benchmarks.to_csv('../output/benchmarks.csv', index=False)  


In [17]:
def dataframe_to_json(df):
    # Transform DataFrame to the specified JSON structure  
    transformed_data = {  
        'baseline': df[['filename', 'class', 'difficulty', 'cl_baseline', 'score_baseline', 'time_baseline']].rename(columns={  
            'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_baseline': 'cl',  
            'score_baseline': 's',  
            'time_baseline': 't'  
        }).to_dict(orient='records'),  
        'baseline-ei': df[['filename', 'class', 'difficulty', 'cl_ei_baseline', 'score_ei_baseline', 'time_ei_baseline']].rename(columns={  
             'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_ei_baseline': 'cl',  
            'score_ei_baseline': 's',  
            'time_ei_baseline': 't'  
        }).to_dict(orient='records'),  
        'efficientad': df[['filename', 'class', 'difficulty', 'cl_efficientad', 'score_efficientad', 'time_efficientad', 'result_img_path']].rename(columns={  
            'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_efficientad': 'cl',  
            'score_efficientad': 's',  
            'time_efficientad': 't',  
            'result_img_path': 'r'  
        }).to_dict(orient='records'),  
        'fomoad': df[['filename', 'class', 'difficulty', 'cl_fomodad', 'score_fomoad', 'time_fomoad']].rename(columns={  
            'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_fomodad': 'cl',  
            'score_fomoad': 's',  
            'time_fomoad': 't'  
        }).to_dict(orient='records')
    }
    
    return  transformed_data

t1 =  dataframe_to_json(dataset_1)
t2 =  dataframe_to_json(dataset_2)
t3 =  dataframe_to_json(dataset_3)

stats_for_website = {
    "cookies_1": t1,
    "cookies_2": t2,
    "cookies_3": t3,
}
import json  
with open('../output/stats_for_website.json', 'w') as outfile:  
    json.dump(stats_for_website, outfile)  